# Julia 機器學習：LIBSVM 支撐向量機

## 作業 029：

請使用 SVM 來建立一個預測模型，來對手寫數字辨識資料集，建立一個分類模型，預測數字為何。

本作業需要使用到的套件有 MLDatasets，請在執行以下範例前先安裝。

```
] add MLDatasets
```

In [1]:
using LIBSVM, MLDatasets, Statistics

## 讀取資料

呼叫 MNIST 資料集的過程中，會先去檢查以前是否有下載過，如果是第一次下載，過程中會詢問是否下載資料集，請回答 `y`。

In [2]:
train_X, train_y = MNIST.traindata(1:10000)
test_X,  test_y  = MNIST.testdata(1:1000)

(FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; … ; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8]

FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; … ; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8]

FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; … ; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8]

...

FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; … ; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8]

FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; … ; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8]

FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N

## 資料轉換

In [3]:
train_X = Matrix(reshape(train_X,(10000,:)))
test_X = Matrix(reshape(test_X,(1000,:)))

1000×784 Array{N0f8,2} with eltype FixedPointNumbers.Normed{UInt8,8}:
 0.0    0.0  0.0    0.0    0.0    0.0    …  0.0    0.0    0.0    0.0    0.0
 0.0    0.0  0.671  0.0    0.0    0.0       0.0    0.0    0.0    0.0    0.502
 0.0    0.0  0.996  0.0    0.0    0.408     0.0    0.0    0.0    0.0    0.988
 0.0    0.0  0.318  0.0    0.0    0.996     0.0    0.0    0.0    0.0    0.988
 0.0    0.0  0.0    0.0    0.0    0.62      0.173  0.0    0.0    0.992  0.988
 0.0    0.0  0.0    0.0    0.0    0.0    …  0.898  0.0    0.0    0.984  0.992
 0.0    0.0  0.0    0.282  0.0    0.0       0.992  0.0    0.0    0.984  0.988
 0.0    0.0  0.0    0.682  0.0    0.0       0.494  0.0    0.0    0.122  0.988
 0.0    0.0  0.0    0.984  0.0    0.0       0.0    0.0    0.0    0.0    0.988
 0.0    0.0  0.0    0.678  0.0    0.0       0.0    0.0    0.0    0.0    0.992
 0.0    0.0  0.0    0.278  0.0    0.0    …  0.0    0.0    0.0    0.0    0.988
 0.0    0.0  0.0    0.282  0.0    0.0       0.0    0.0    0.0    0.0    0.

In [4]:
train_y = string.(train_y)
test_y = string.(test_y)

1000-element Array{String,1}:
 "7"
 "2"
 "1"
 "0"
 "4"
 "1"
 "4"
 "9"
 "5"
 "9"
 "0"
 "6"
 "9"
 ⋮
 "1"
 "6"
 "2"
 "3"
 "9"
 "0"
 "1"
 "2"
 "2"
 "0"
 "8"
 "9"

## 預測

In [5]:
model = LIBSVM.fit!(SVC(), train_X, train_y)

ŷ = String[]
for i = 1:size(test_X, 1)
    y_ = LIBSVM.predict(model, Matrix(test_X[i, :]'))
    append!(ŷ, y_)
end

## 評估

In [9]:
accuracy() = mean((ŷ .== test_y))*100

accuracy (generic function with 1 method)

In [10]:
accuracy()

12.6